# Getting Started 
This section is based on the [Datathon Tutorial](https://colab.research.google.com/github/GoogleCloudPlatform/healthcare/blob/master/datathon/anzics18/tutorial.ipynb#scrollTo=BFOlLhuTE9OG) by Google. 

Import all necessary Python libraries. 

In [ ]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import uuid
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import gensim
from gensim.models.doc2vec import TaggedDocument
import random
import time
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from scipy.sparse import hstack

# Imports for accessing data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

Provide your credentials to the runtime

In [ ]:
auth.authenticate_user()

# Accessing the MIMIC-III Dataset

This section assumes that you have already followed [these instructions](https://mimic.mit.edu/docs/iii/tutorials/intro-to-mimic-iii-bq/) to get access to the MIMIC-III dataset in your personal GCP BigQuery instance. 

The data-hosting project `physionet-data` has read-only access. As a result, you need to set a default project that you have BigQuery access to. This notebook uses `cse-6250-project`. **You should create a project of this name in your Google Cloud instance or change the below project name to one that you have write access to.**

In [ ]:
#@title Setting default project ID (you may need to create your own) {display-mode:"both"}
project_id='cse-6250-project' # @param
client = bigquery.Client(project=project_id)
datasets = client.list_datasets()
for dataset in datasets:
  did = dataset.dataset_id
  # Optional to verify that you are able to access the datasets.
  # The datasets we care about are mimiciii_clinical and mimiciii_notes.
  print(did) 

os.environ["GOOGLE_CLOUD_PROJECT"]=project_id
!gcloud config set project $project_id

## Data Exploration

1. Get basic statistics on the data
2. Create a histogram on note length
3. Create a histogram of note category

In [ ]:
# Create a histogram of category
query = "SELECT CATEGORY, count(*) AS count FROM `mimiciii_notes.note_events` GROUP BY CATEGORY"
category_df = client.query(query).to_dataframe()

category_df.plot.bar(x="CATEGORY", y="count")
plt.xlabel('Category')
plt.ylabel('Count')
plt.show()

In [ ]:
# Create a histogram of note length
query = "SELECT LENGTH(TEXT) AS length FROM `mimiciii_notes.note_events`"
length_df = client.query(query).to_dataframe()

plt.hist(length_df['length'], bins = 20)
plt.xlabel('Note Length')
plt.ylabel('Count')
plt.show()

In [ ]:
query = "SELECT TEXT FROM `mimiciii_notes.note_events` WHERE RAND() < 3/(SELECT COUNT(*) FROM `mimiciii_notes.note_events`)"
temp = client.query(query)
notes = temp.to_dataframe()
print(notes)

## Feature Extraction with BigQuery


`ADMISSIONS`
1.   Remove all newborn records ✅
2.   Remove records that have a death date ✅
3.   For each subject ID, add a new column to contain the next visit (basically linking each visit to the next visit. Will be used for prediction step) ✅
4.   Filter out the ELECTIVE next admissions and backfilling ✅
5.   Calculate the days until the next admission ✅
6.   Create a chart of the number of days between admissions for the report ✅

In [ ]:
# Drops columns that aren't relevant, removes newborn admissions, and removes records with a death date
# Note: if you are building off of this query, you may actually want to overwrite the existing table. 
# In that case, remove the part that says IF NOT EXISTS below 
table_id = "mimiciii_notes.admissions_cleaned"

query = f"""
CREATE TABLE IF NOT EXISTS 
`{table_id}` (
  `ROW_ID` INTEGER,
  `SUBJECT_ID` INTEGER,
  `HADM_ID` INTEGER,
  `ADMITTIME` DATETIME,
  `DISCHTIME` DATETIME,
  `DEATHTIME` DATETIME,
  `ADMISSION_TYPE` STRING
)
AS SELECT ROW_ID,SUBJECT_ID, HADM_ID, ADMITTIME, DISCHTIME, DEATHTIME, ADMISSION_TYPE 
 FROM `mimiciii_notes.admissions` 
 WHERE ADMISSION_TYPE != 'NEWBORN'
 AND DEATHTIME IS NULL
"""

qry = client.query(query)#.to_dataframe()
assert qry.exception() is None

table = client.get_table(table_id)
assert table is not None

print("Number of records: " + str(table.num_rows))

In [ ]:
# For each record get the fields from the next visit
# And calculate the number of days til the next admission
table_id = "mimiciii_notes.admissions_next"

query = f"""
CREATE OR REPLACE TABLE
`{table_id}` (
  `ROW_ID` INTEGER,
  `SUBJECT_ID` INTEGER,
  `HADM_ID` INTEGER,
  `ADMITTIME` DATETIME,
  `DISCHTIME` DATETIME,
  `DEATHTIME` DATETIME,
  `ADMISSION_TYPE` STRING,
  `NEXT_ADMIT_ID` INTEGER,
  `NEXT_ADMIT_TIME` DATETIME,
  `NEXT_ADMIT_TYPE` STRING,
  `DAY_DIFF` INTEGER
)
AS 
(
  SELECT x.*
    ,a.ADMITTIME AS NEXT_ADMIT_TIME
    ,a.ADMISSION_TYPE AS NEXT_ADMIT_TYPE
    ,DATE_DIFF(a.DISCHTIME, x.ADMITTIME, DAY)
  FROM
  (
    SELECT a1.ROW_ID, a1.SUBJECT_ID, a1.HADM_ID, a1.ADMITTIME, a1.DISCHTIME, a1.DEATHTIME, a1.ADMISSION_TYPE
      ,ARRAY_AGG(a2.HADM_ID ORDER BY  a2.ADMITTIME LIMIT 1)[SAFE_OFFSET(0)] AS NEXT_ADMIT_ID
    FROM `mimiciii_notes.admissions_cleaned` a1
    LEFT JOIN `mimiciii_notes.admissions_cleaned` a2 ON a1.SUBJECT_ID = a2.SUBJECT_ID AND a2.ADMISSION_TYPE NOT IN ('ELECTIVE','NEWBORN') AND a2.ADMITTIME > a1.ADMITTIME
    GROUP BY a1.ROW_ID, a1.SUBJECT_ID, a1.HADM_ID, a1.ADMITTIME, a1.DISCHTIME, a1.DEATHTIME, a1.ADMISSION_TYPE
  ) x
  LEFT JOIN `mimiciii_notes.admissions_cleaned` a ON x.NEXT_ADMIT_ID = a.HADM_ID
)


"""

qry = client.query(query)#.to_dataframe()
assert qry.exception() is None

table = client.get_table(table_id)
assert table is not None

print("Number of records: " + str(table.num_rows))

In [ ]:
# Create a histogram of the number of days between admissions
# Excludes the null values and limit to a year for readability
query = "SELECT * FROM `mimiciii_notes.admissions_next`"
dataframe = (client.query(query).result().to_dataframe())
plt.hist(dataframe.loc[~dataframe.DAY_DIFF.isnull(),'DAY_DIFF'], bins=range(0,365,25))
plt.xlim([0,365])
plt.xlabel('Days til Next Admission')
plt.ylabel('Count')
plt.show()

`NOTE_EVENTS`


1.   Deal with duplicates—multiple records per patient, per visit (see the linked paper for a more thorough description) ✅
2.   Convert all notes to lower case ✅
3.   Remove new line and return characters in the notes ✅
4.   Replace any punctuation with a space ✅



In [ ]:
# There are multiple notes for each admission for each patient
# Combine all the notes into a single text
table_id = "mimiciii_notes.note_events_cleaned"

query = f"""
CREATE OR REPLACE TABLE
`{table_id}` (
  `SUBJECT_ID` INTEGER,
  `HADM_ID` INTEGER,
  `TEXT` STRING
)
AS (
  SELECT n.SUBJECT_ID
    ,n.HADM_ID
    ,string_agg(n.TEXT, " ")
  FROM `mimiciii_notes.note_events` n
  WHERE n.CATEGORY = 'Discharge summary'
  GROUP BY n.SUBJECT_ID, n.HADM_ID
  )
 
"""

qry = client.query(query)#.to_dataframe()
assert qry.exception() is None

table = client.get_table(table_id)
assert table is not None

print("Number of records: " + str(table.num_rows))

Feature Engineering

1.    Combine the note events and admissions tables ✅
2.    Finish any necessary data cleaning for the notes in pandas ✅
3.    Create a field for the target variable - 1 if readmitted within **45** days and 0 otherwise ✅

In [ ]:
# Join the note events and admission tables
# Remove any admissions that don't have notes associated with it
table_id = "mimiciii_notes.admissions_notes_comb"

query = f"""
CREATE OR REPLACE TABLE
`{table_id}` (
  `SUBJECT_ID` INTEGER,
  `HADM_ID` INTEGER,
  `DAY_DIFF` INTEGER,
  `TEXT` STRING
)
AS (
  SELECT a.SUBJECT_ID
    ,a.HADM_ID
    ,a.DAY_DIFF
    ,n.TEXT
  FROM `mimiciii_notes.admissions_next` a
  INNER JOIN `mimiciii_notes.note_events_cleaned` n ON a.HADM_ID  = n.HADM_ID
  )
"""

qry = client.query(query)#.to_dataframe()
assert qry.exception() is None

table = client.get_table(table_id)
assert table is not None

print("Number of records: " + str(table.num_rows))

In [ ]:
# Save to a pandas dataframe
query = "SELECT * FROM `mimiciii_notes.admissions_notes_comb`"
df = (client.query(query).result().to_dataframe())
print(len(df))
print(df.info)

In [ ]:
# Finish data cleaning of the notes
# Convert all notes to lower case
df['TEXT'] = df['TEXT'].str.lower()
# Remove new line and return characters or punctuation with a space
df['TEXT'] = df['TEXT'].str.replace('\n', ' ')
df['TEXT'] = df['TEXT'].str.replace('\r', ' ')
df['TEXT'] = df['TEXT'].str.replace('\W', ' ')

In [ ]:
# Creature the target variable - 1 if the days between admissions is less than 45 and 0 otherwise
df['TARGET'] = np.where(df['DAY_DIFF'] <= 45, 1, 0)

In [ ]:
#save df to google drive to save time
saveFilePickle = False
if saveFilePickle:
  from google.colab import drive
  import pickle
  drive.mount('/content/drive/')
  df.to_pickle('/content/drive/MyDrive/CSE6250/df.pkl')


## NLP


In [ ]:
import string
stop_words = ['the','and','to','of','was','with','a','on','in','for','name',
            'is','patient','s','he','at','as','or','one','she','his','her','am',
            'were','you','pt','pm','by','be','had','your','this','date',
            'from','there','an','that','p','are','have','has','h','but','o',
            'namepattern','which','every','also','should','if','it','been','who','during']

def tokenize_note(text):

    def tokenizer_helper(text):        
        remove_list = string.punctuation+'0123456789'
        t = str.maketrans(dict.fromkeys(remove_list, " "))
        text = text.lower().translate(t)
        tokens = nltk.word_tokenize(text)
        tokens = [token for token in tokens if token not in stop_words]
        return tokens

    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in tokenizer_helper(sent):
            if len(word) >= 2:
              tokens.append(word.lower())
    return tokens



## Free up space before modeling

In [ ]:
#deleting unused variables and free up RAM
import gc, sys,psutil, pickle
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)
for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))
print('Memory in Gb', get_memory_usage())

In [ ]:
del dataframe
gc.collect()

## NLP and Modeling


In [ ]:
class model():
    def __init__(self, df, textColumn = 'TEXT', target = 'TARGET', SEED = 2):
        random.seed(SEED)
        self.SEED = SEED
        self.df = df.copy()
      
        #column with text
        self.textColumn = textColumn
    
    #lem and remove stopwords
    def lem(self, phrase):
        stop_words_ = list(set(stopwords.words('english')))
        wnl=WordNetLemmatizer()
        lemmed=[]
        for i in word_tokenize(phrase):
            if i not in (stop_words_) and len(i) >= 2:
                w1=wnl.lemmatize(i, 'v')
                w2=wnl.lemmatize(w1, 'a')
                w3=wnl.lemmatize(w2, 'n')
                lemmed.append(w3.lower())
        return " ".join(lemmed)


        #return " ".join([wnl.lemmatize(words.lower(), pos="v") for words in word_tokenize(phrase) if (words not in (stop_words_))])
    
    
    
    #remove spaces and underscores
    def remove_spaces(self, phrase):
        return  " ".join(phrase.replace('_', '').split())


    #adding word count as feature
    def add_wordCount(self):
        self.df['countLen']  = [len(tokenize_note(i)) for i in df[self.textColumn]]

    def downsample (self, X, y):
        # imbalanced classes so need to deal with that
        # easiest and quickest way to deal with this is to sub-sample the negative records and create a balanced dataset
        undersample = RandomUnderSampler(sampling_strategy=0.5)
        X_under, y_under = undersample.fit_resample(X.values.reshape(-1,1), y)
        return X_under, y_under


    def preprocess(self, add_wordLen = False, downsample = False):
        print('--------------------------------Sample Text-------------')
        for i in self.df[self.textColumn].head(1):
            print(i)
        
      

        #30% test, 70% train
        self.X_train, self.X_test, self.y_train, self.y_test  =train_test_split(self.df[self.textColumn], self.df['TARGET'], test_size=0.3, random_state=self.SEED, stratify=self.df['TARGET'])


        print('------------------------------Start Downsamping--------------------------')
        self.X_train, self.y_train = self.downsample(self.X_train, self.y_train)
        print('-----------------------------Finished Downsamping----------------------')
        #self.X_train, self.X_val, self.y_train, self.y_val  = train_test_split(self.X_train, self.y_train, test_size=0.25, random_state=1, stratify=self.y_train)
        


        #remove spaces and underscores
        self.X_train = [self.remove_spaces(x) for x in self.X_train.squeeze()]
        self.X_test = self.X_test.apply(lambda x: self.remove_spaces(x))
        print('------------------Sample Text after removing spaces and underscores----------------')
        for i in self.X_train[:1]:
            print(i)

        print('---------------------------Start Lemmatization--------------------------------')
        self.X_train = [self.lem(x) for x in self.X_train]
        self.X_test = self.X_test.apply(lambda x: self.lem(x))
        print('---------------------------------finished Lemmatize--------------------------')



        print('Start Vectorizing')
        #vectorize
        vect_word = TfidfVectorizer(max_features=60000, lowercase=True, analyzer='word',tokenizer=tokenize_note,ngram_range=(1,3),dtype=np.float32)
        self.tr_vect = vect_word.fit_transform(self.X_train)
        self.vl_vect = vect_word.transform(self.X_test)
        print('Finished Vectorizing')


        print(self.tr_vect.shape[1] , 'total Width of the train vector' )


In [ ]:
SEED = 2
textColumnToUse = 'TEXT'
target= 'TARGET'
models = model(df, textColumnToUse, target, SEED)
models.preprocess()



In [ ]:
# Best Model - Logistic Regression

lgr = LogisticRegression(solver='saga',multi_class='multinomial', max_iter=1000, C=1,verbose = True, n_jobs = -1, class_weight='balanced', random_state = models.SEED).fit(models.tr_vect, models.y_train)
t = time.process_time()
lgr_y_pred = lgr.predict(models.vl_vect)
elapsed_time = time.process_time() - t
print('Total elappsed time: ', elapsed_time)
print('LGR AUC:', metrics.roc_auc_score(models.y_test, lgr_y_pred))
print("LGR Accuracy:",metrics.accuracy_score(models.y_test, lgr_y_pred))
print("LGR Precision:",metrics.precision_score(models.y_test, lgr_y_pred))
print("LGR Recall:",metrics.recall_score(models.y_test, lgr_y_pred))

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay.from_predictions(models.y_test, lgr_y_pred, cmap=plt.cm.Blues, display_labels=['Not Readmitted','Readmitted'])
disp.ax_.set_title('Non-Normalized Confusion Matrix')
plt.show()

disp = ConfusionMatrixDisplay.from_predictions(models.y_test, lgr_y_pred, cmap=plt.cm.Blues, display_labels=['Not Readmitted','Readmitted'], normalize = 'all')
disp.ax_.set_title('Normalized Confusion Matrix')
plt.show()

In [ ]:
# top features
vect_word = TfidfVectorizer(max_features=60000, lowercase=True, analyzer='word',tokenizer=tokenize_note,ngram_range=(1,3),dtype=np.float32)
tr_vect = vect_word.fit_transform(models.X_train)
feature_names = np.array(vect_word.get_feature_names())
importance_ind = np.argsort(np.asarray(tr_vect.sum(axis=0)).ravel())[::-1]
importance_val = np.sort(np.asarray(tr_vect.sum(axis=0)).ravel())[::-1]


In [ ]:
n = 50
top_words = feature_names[importance_ind[:n]]
top_scores = importance_val[:n]
bottom_words = feature_names[importance_ind[-n:]]
bottom_scores = importance_val[-n:]

y_position = np.arange(len(top_words))
fig = plt.figure(figsize=(10, 10))  

plt.subplot(121)
plt.barh(y_position,bottom_scores, align='center', alpha=0.5)
plt.title('Least Relevant', fontsize=20)
plt.yticks(y_position, bottom_words, fontsize=14)
plt.xlabel('Importance Score', fontsize=20)

plt.subplot(122)
plt.barh(y_position,top_scores, align='center', alpha=0.5)
plt.title('Most Relevant', fontsize=20)
plt.yticks(y_position, top_words, fontsize=14)
plt.xlabel('Importance Score', fontsize=20)

plt.subplots_adjust(wspace=1)
plt.show()
